In [1]:
import osmnx as ox
import os
import geopandas as gpd
from shapely.geometry import Point
import matplotlib.pyplot as plt
import pandas as pd
from pyproj import CRS, Transformer

In [2]:
def get_india_states():
    current_dir = os.getcwd()
    parent_dir = os.path.dirname(current_dir)
    aoi_dir = os.path.join(parent_dir, 'aoi')

    india_path = os.path.join(aoi_dir, 'india1.json')
    india_states = gpd.read_file(india_path)
    return india_states

def get_lakes(longitude: float, latitude: float, distance: int, area: int = 1000000) -> gpd.GeoDataFrame:
    
    tags = {'natural': 'water'}  # Searching for natural water bodies
    
    waterbodies = ox.features_from_point((latitude, longitude), tags=tags, dist=distance)
    allowed_values = ['pond', 'reservoir', 'lake', 'basin']
    
    lakes = waterbodies[waterbodies['water'].isin(allowed_values)]
    lakes = lakes[['name', 'geometry', 'water']]

    # Ensure geometries are valid and not empty
    lakes = lakes[lakes['geometry'].notna()]
    lakes = lakes[lakes.geometry.is_valid]

    # Add centroid coordinates
    lakes["geometry_long_lat"] = lakes["geometry"]

    # Determine the appropriate UTM zone and reproject
    zone_number = int((longitude + 180) / 6) + 1
    utm_crs = CRS(f'EPSG:326{zone_number}')
    lakes = lakes.to_crs(utm_crs)
    
    transformer = Transformer.from_crs('EPSG:4326', utm_crs, always_xy=True)
    reference_point_utm = transformer.transform(longitude, latitude)
    
    lakes['area'] = lakes.geometry.area
    lakes['centroid'] = lakes.geometry.centroid

    
    lakes = lakes.sort_values(by='area', ascending=False)
    lakes['distance'] = lakes.apply(calculate_distance, axis =1, point = reference_point_utm)
    lakes['centroid_ws'] = lakes['centroid'].to_crs(epsg = 4326)

    lakes = lakes[lakes['area']>area]

    lakes = lakes[[ 'name', 'geometry_long_lat', 'area', 'distance', 'centroid_ws', 'centroid']]
    lakes = lakes.reset_index()
    return lakes

def get_all_lakes(df, distance, area = 1000000, drop_duplicated = True):
    dataframes = []
    for i, row in df.iterrows():
        longitude = row["longitude"]
        latitude = row["latitude"]
        university_lakes = get_lakes(longitude, latitude, distance, area)
        dataframes.append(university_lakes)
        university_lakes['point name'] = row["name"]

    all_lakes = pd.concat(dataframes, ignore_index=True)
    all_lakes = all_lakes.sort_values(by='area', ascending=False)
    plot_indian_lakes(df, all_lakes)
    if drop_duplicated == True:
        return drop_duplicated_lakes(all_lakes)
    return all_lakes



def calculate_distance(row, point):
    centroid_utm = row['centroid']
    return centroid_utm.distance(Point(point))

def calculate_crs(point: Point)->int:
    longitude = point.coords[0][1]
    zone_number = int((longitude + 180) / 6) + 1
    utm_crs = int(f'326{zone_number}')
    return utm_crs

def transform_geometries_by_crs(lakes: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    # Crear una lista para almacenar las geometrías transformadas
    transformed_geometries = []

    for idx, row in lakes.iterrows():
        # Obtener el CRS para el registro actual
        crs_code = row['calculated_crs']
        
        try:
            # Crear un CRS a partir del código EPSG
            crs = CRS(f'EPSG:{int(crs_code)}')

            # Crear un GeoDataFrame temporal con el CRS actual
            temp_gdf = gpd.GeoDataFrame({'geometry': [row['geometry']]}, crs='EPSG:4326')
            temp_gdf = temp_gdf.to_crs(crs)

            # Obtener la geometría transformada
            transformed_geometry = temp_gdf.iloc[0].geometry
        except Exception as e:
            print(f"Error al transformar la geometría en el índice {idx}: {e}")
            transformed_geometry = row['geometry']  # Dejar la geometría sin transformar en caso de error

        # Agregar la geometría transformada a la lista
        transformed_geometries.append(transformed_geometry)

    lakes_transformed = lakes.copy()
    lakes_transformed['geometry'] = transformed_geometries

    return lakes_transformed

def drop_duplicated_lakes(gdf):
    idx = gdf.groupby('osmid')['distance'].idxmin()
    filtered_lakes = gdf.loc[idx]

    # Resetear el índice del DataFrame resultante
    filtered_lakes = filtered_lakes.reset_index(drop=True)
    return filtered_lakes

def plot_indian_lakes(universities, lakes):
    india_states = get_india_states()
    universities['geometry'] = universities.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
    universities_gdp = gpd.GeoDataFrame(universities, geometry = 'geometry')

    fig, ax = plt.subplots(1, 1, figsize=(10, 10))

    # Plotear el mapa base de la India
    india_states.plot(ax=ax, color = 'lightgrey', edgecolor='black' )
    lakes['centroid_ws'].plot(ax=ax, color='blue', markersize=50, label='Lakes')
    
    universities_gdp.plot(ax=ax, color='red', markersize=50, label='Universities')
    plt.legend()

    # Añadir título
    plt.title('Lakes and Universities from India')

    # Mostrar el plot
    plt.show()

In [16]:
lakes = get_lakes(76.8, 30.73, 2000)
sukhna_polygon = lakes[lakes['name']=='Sukhna Lake']['geometry_long_lat'][0]

coords = list(sukhna_polygon.exterior.coords)

geojson = {
    "type": "Polygon",
    "coordinates": [coords]
}

gdf = gpd.GeoDataFrame([{'geometry': sukhna_polygon}], crs="EPSG:4326")

gdf.to_file("../aoi/sukhna_polygon.geojson", driver="GeoJSON")

